<a href="https://colab.research.google.com/github/abhirajr219/Cocoder_movie_genre-Predictor/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install colabcode

!pip install wikipedia

!pip install pyngrok nest_asyncio fastapi uvicorn loguru

In [13]:
!pip install typing

     |████████████████████████████████| 78 kB 4.2 MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26325 sha256=65eee28e830f947a69d07b06905ffe4ef7df68120a45c12ed1aed1bd26af354b
  Stored in directory: /root/.cache/pip/wheels/35/f3/15/01aa6571f0a72ee6ae7b827c1491c37a1f72d686fd22b43b0e
Successfully built typing


In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install sklearn

In [4]:
!pip install pydantic

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("26ByyAjyiwDZ6S19WQd4y8gXwBh_6ztjmj4cFd68UxEAuH6hC")

In [5]:
from colabcode import ColabCode
from fastapi import FastAPI

In [6]:
cc=ColabCode(port=12000,code=False)

In [62]:
 %%writefile models.py
 from pydantic import BaseModel,conlist
 from typing import List

 class request_body(BaseModel):
   data: List[conlist(str,min_items=1,max_items=1)]

Overwriting models.py


In [73]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import pickle
import logging
from models import request_body
import wikipedia
import numpy as np
app=FastAPI(title="Movie Genre Preditor")

my_logger = logging.getLogger()
my_logger.setLevel(logging.DEBUG)
logging.basicConfig(level=logging.DEBUG, filename='logs.log')


model=None

@app.on_event("startup")
def load_model():
  global model
  model=pickle.load(open("/content/drive/MyDrive/AI/movie1.pkl","rb"))
@app.get("/readroot")
async def Owners():
  return {"Owners ":"Jayalekshmi & Abhiraj"}

@app.post("/", tags=["prediction"])
async def get_prediction(iris: request_body):
    data=dict(iris)['data']  
    print(iris)
    try:
        try:
             plot=wikipedia.page(data[0][0]+" "+"movie").content[0:5500]
             print(plot,data)
        except: 
             plot=wikipedia.page(data[0][0]).content[0:5500]
             print(plot,data)
    
        def genre_prediction(sample_script):
              sample_script = re.sub(pattern='[^a-zA-Z]',repl=' ', string=sample_script)
              sample_script = sample_script.lower()
              sample_script_words = sample_script.split()
              sample_script_words = [word for word in sample_script_words if not word in set(stopwords.words('english'))]
              ps = PorterStemmer()
              final_script = [ps.stem(word) for word in sample_script_words]
              final_script = ' '.join(final_script)
              from sklearn.feature_extraction.text import CountVectorizer
              cv = CountVectorizer(max_features=300, ngram_range=(1,2))
              temp = cv.fit_transform([final_script]).toarray()
              return temp

        test_value=genre_prediction(plot)
        value=(model.predict(test_value))
        genre_mapper = {'other': 0, 'action': 1, 'adventure': 2, 'comedy':3,'drama':4, 'horror':5, 'romance':6, 'sci-fi':7, 'thriller': 8}

        return('Prediction: {}'.format(list(genre_mapper.keys())[value[0]]))
  
    except:
      
        return {"prediction": "Something went wrong in finding the movie!.Try giving more specific name of the movie/add word <Movie> to the Search data or try adding year!!!"}


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://aed3-35-197-125-10.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [685]
INFO:     Waiting for application startup.
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.23.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     103.149.158.107:0 - "GET / HTTP/1.1" 405 Method Not Allowed
INFO:     103.149.158.107:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     103.149.158.107:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     103.149.158.107:0 - "GET /openapi.json HTTP/1.1" 200 OK
data=[['The Hangover Part II']]
The Hangover Part II (also known as The Hangover 2: The Wolfpack is Back or The Hangover Again in the working title) is a 2011 American comedy film produced by Legendary Pictures and distributed by Warner Bros. Pictures. It is the sequel to the 2009 film The Hangover and the second installment in The Hangover trilogy. Directed by Todd Phillips, who co-wrote the script with Craig Mazin and Scot Armstrong, the film stars Bradley Cooper, Ed Helms, Zach Galifianakis, Ken Jeong, Jeffrey Tambor, Justin Bartha, and Paul Giamatti.
It tells the story of Phil, Stu, Alan, and Doug as they travel to Thailand for Stu's wedding. After the bachelor party in Las Vegas, Stu takes no chances and opts for a safe